In [20]:
from typing import NamedTuple
from search_utils import AbstractNode


class State(NamedTuple):
    a: int
    b: int
    c: int

    def __repr__(self) -> str:
        return f"State(a={self.a}, b={self.b}, c={self.c})"

    def __str__(self) -> str:
        return self.__repr__()

    def __eq__(self, __value: object) -> bool:
        if not isinstance(__value, State):
            return False

        return self.a == __value.a and self.b == __value.b and self.c == __value.c

    def __hash__(self) -> int:
        return hash((self.a, self.b, self.c))

    def is_within_boundaries(self, boundary: "State"):
        return self.a <= boundary.a and self.b <= boundary.b and self.c <= boundary.c

    def copy_with(self, **kwargs):
        return State(**{**self._asdict(), **kwargs})


class Node(AbstractNode[State]):
    def __init__(self, action: str, state: State, parent: State | None = None) -> None:
        super().__init__(action, state, parent)

    def __eq__(self, __value: object) -> bool:
        if not isinstance(__value, Node):
            return False

        return self.state == __value.state

    def __hash__(self) -> int:
        return hash(self.state)

In [21]:
CandidateAction = tuple[str, State]


def create_candidates(current_state: State):
    a, b, c = current_state

    candidates: list[CandidateAction] = [
        ("A plays with B", State(a + 1, b + 1, c)),
        ("A plays with C", State(a + 1, b, c + 1)),
        ("B plays with C", State(a, b + 1, c + 1)),
    ]

    return candidates

In [22]:
from typing import Literal, Callable
from search_utils import StackFrontier, QueueFrontier, reconstruct_path


SearchAlgorithm = Literal["dfs", "bfs"]
Heuristic = Callable[[Node], float]


class PingPong:
    def __init__(self, initial_state: State, goal_state: State) -> None:
        self.root = initial_state
        self.goal = goal_state

    def __neighbors(self, parent: Node):
        candidates = create_candidates(parent.state)
        possible_neighbors: list[Node] = []

        for action, state in candidates:
            if state.is_within_boundaries(self.goal) and action != parent.action:
                possible_neighbors.append(Node(action, state, parent))

        return possible_neighbors

    def solve(self, algorithm: SearchAlgorithm = "dfs"):
        num_explored = 0
        explored: set[State] = set()
        frontier = None

        match algorithm:
            case "bfs":
                frontier = QueueFrontier[State]()
            case "dfs":
                frontier = StackFrontier[State]()
            case _:
                raise ValueError(
                    f"{algorithm} is not a valid value for the search algorithm. You must choose 'dfs' or 'bfs'"
                )

        frontier.add(Node("Game start", self.root))

        while not frontier.is_empty:
            current_node = frontier.remove()
            num_explored += 1

            print(
                f"Itération N°{num_explored} -> Action = {current_node.action} -> {current_node.state}"
            )

            if current_node.state == self.goal:
                return reconstruct_path(current_node), num_explored

            explored.add(current_node.state)

            for neighbor in self.__neighbors(current_node):
                if neighbor.state not in explored and neighbor not in frontier:
                    frontier.add(neighbor)

        raise Exception("No solution found")

    def solve_with_heuristic(self, heuristic: Heuristic):
        num_explored = 0
        explored: set[State] = set()
        frontier: list[Node] = []

        frontier.append(Node("Game start", self.root))

        while len(frontier):
            current_node = frontier.pop(0)
            num_explored += 1

            print(
                f"Itération N°{num_explored} -> Action = {current_node.action} -> {current_node.state}"
            )

            if current_node.state == self.goal:
                return reconstruct_path(current_node), num_explored

            explored.add(current_node.state)

            for neighbor in self.__neighbors(current_node):
                if neighbor.state not in explored and neighbor not in frontier:
                    frontier.append(neighbor)

            # Choose the closest node
            frontier = sorted(frontier, key=heuristic)

        raise Exception("No solution found")

In [23]:
initial_state = State(0, 0, 0)
goal = State(10, 15, 17)
problem = PingPong(initial_state, goal)
solution, num_explored = problem.solve("dfs")

Itération N°1 -> Action = Game start -> State(a=0, b=0, c=0)
Itération N°2 -> Action = B plays with C -> State(a=0, b=1, c=1)
Itération N°3 -> Action = A plays with C -> State(a=1, b=1, c=2)
Itération N°4 -> Action = B plays with C -> State(a=1, b=2, c=3)
Itération N°5 -> Action = A plays with C -> State(a=2, b=2, c=4)
Itération N°6 -> Action = B plays with C -> State(a=2, b=3, c=5)
Itération N°7 -> Action = A plays with C -> State(a=3, b=3, c=6)
Itération N°8 -> Action = B plays with C -> State(a=3, b=4, c=7)
Itération N°9 -> Action = A plays with C -> State(a=4, b=4, c=8)
Itération N°10 -> Action = B plays with C -> State(a=4, b=5, c=9)
Itération N°11 -> Action = A plays with C -> State(a=5, b=5, c=10)
Itération N°12 -> Action = B plays with C -> State(a=5, b=6, c=11)
Itération N°13 -> Action = A plays with C -> State(a=6, b=6, c=12)
Itération N°14 -> Action = B plays with C -> State(a=6, b=7, c=13)
Itération N°15 -> Action = A plays with C -> State(a=7, b=7, c=14)
Itération N°16 -> 

In [24]:
from search_utils import draw_path

draw_path(solution[-1], rankdir="LR")

## Solving the problem with greedy best first search

In [25]:
import math

# Defining a set of heuristic functions


def arithmetic_mean(node: Node):
    return sum(node.state) / len(node.state)


def geometric_mean(node: Node):
    return math.pow(math.prod(node.state), 1 / len(node.state))


def harmonic_mean(node: Node):
    reciprocal_sum = sum(1 / x for x in node.state if x != 0)

    if reciprocal_sum == 0:
        return 0

    return len(node.state) / reciprocal_sum

In [26]:
solution, num_explored = problem.solve_with_heuristic(heuristic=geometric_mean)
draw_path(solution[-1], rankdir="LR")

Itération N°1 -> Action = Game start -> State(a=0, b=0, c=0)
Itération N°2 -> Action = A plays with B -> State(a=1, b=1, c=0)
Itération N°3 -> Action = A plays with C -> State(a=2, b=1, c=1)
Itération N°4 -> Action = B plays with C -> State(a=2, b=2, c=2)
Itération N°5 -> Action = A plays with B -> State(a=3, b=3, c=2)
Itération N°6 -> Action = A plays with C -> State(a=4, b=3, c=3)
Itération N°7 -> Action = B plays with C -> State(a=4, b=4, c=4)
Itération N°8 -> Action = A plays with B -> State(a=5, b=5, c=4)
Itération N°9 -> Action = A plays with C -> State(a=6, b=5, c=5)
Itération N°10 -> Action = B plays with C -> State(a=6, b=6, c=6)
Itération N°11 -> Action = A plays with B -> State(a=7, b=7, c=6)
Itération N°12 -> Action = A plays with C -> State(a=8, b=7, c=7)
Itération N°13 -> Action = B plays with C -> State(a=8, b=8, c=8)
Itération N°14 -> Action = A plays with B -> State(a=9, b=9, c=8)
Itération N°15 -> Action = A plays with C -> State(a=10, b=9, c=9)
Itération N°16 -> Acti

Exception: No solution found